In [24]:
import plotly.express as px
import sklearn as sk
import pandas as pd
import numpy as np
import random


In [2]:
actions_data = pd.read_csv("../data/actions.csv").dropna()
cars_data = pd.read_csv("../data/cars_about.csv").dropna()

In [3]:
cars_data

,car_id,car_model,used_label,price,exteriorColor,interiorColor,drivetrain,minMPG,maxMPG,fuelType,transmission,engine,mileage
0,1.0,Toyota Sienna SE,Used,$39,Red,Black,Front-wheel Drive,19.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,29403.0
1,2.0,Ford F-150 Lariat,Used,$49,Shadow Black,Black,Four-wheel Drive,19.0,24.0,Gasoline,10-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,32929.0
2,3.0,RAM 1500 Laramie,Used,$41,Granite Crystal Clearcoat Metallic,Black,Four-wheel Drive,15.0,21.0,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,23173.0
3,4.0,Honda Accord Sport SE,Used,$28,Gray,–,Front-wheel Drive,29.0,35.0,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,10598.0
4,5.0,Lexus RX 350,Used,$49,Eminent White Pearl,Birch,Front-wheel Drive,20.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,28137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,401.0,Jeep Grand Cherokee Limited,Used,$33,Diamond Black,Black,Four-wheel Drive,18.0,25.0,Gasoline,8-Speed Automatic,3.6L V6 24V MPFI DOHC,43807.0
400,402.0,Toyota RAV4 Hybrid XLE,Toyota Certified,$30,Super White,Nutmeg,All-wheel Drive,34.0,30.0,Hybrid,Automatic CVT,2.5L I4 16V MPFI DOHC Hybrid,41944.0
401,403.0,Cadillac XT4 Sport,Used,$35,Black,Black,All-wheel Drive,22.0,29.0,Gasoline,9-Speed Automatic,2.0L I4 16V GDI DOHC Turbo,49386.0
402,404.0,Chevrolet Tahoe LTZ,Used,$35,Tungsten Metallic,Jet Black,Four-wheel Drive,16.0,22.0,Gasoline,6-Speed Automatic,5.3L V8 16V GDI OHV,115795.0


In [4]:
actions_data

,car_id,user_id
0,84,0
1,171,0
2,80,0
3,126,0
4,184,0
...,...,...
2869,13,74
2870,17,74
2871,17,74
2872,16,74


In [5]:
actions_data = actions_data.astype(int)

In [135]:
user_data = pd.DataFrame(
    columns = sorted(actions_data.car_id.unique(), key=int), 
    index=actions_data.user_id.unique()
    )
user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


У меня есть действия пользователей в которые входит только информация о том лайкнул он авто или нет, мне кажется, что это довольно скупая информация. Для того чтобы диверсифицировать предпочтения сгенерированных пользователей я буду рандомно генерировать рейтинги для фильмов <br> Каким образом? <br> авто было добавлено в любимые? => random([3, 4, 5]) <br> С остальными авто поступаю иначе, генерирую для пользователя рандомное кол-во авто с низким рейтингом и уже эти авто раздаю рейтинги от 1-2

In [136]:
for user_id in actions_data.user_id:
    current_user_actions = actions_data.query(f"user_id == {user_id}")
    for car_id in current_user_actions.car_id:
        user_data[int(car_id)].iloc[user_id] = random.randint(4, 5) 

user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5,NaN,NaN,NaN,4,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,NaN,4,4,4,4,4,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5,NaN,5,4,5,5,5,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
num_unique_cars = len(user_data.columns)

for row_idx in user_data.index:
    # specify the number of NaN values to replace with random numbers
    # from 0 to number of non-defined cars
    n = random.randint(0, num_unique_cars - user_data.iloc[row_idx].notna().sum())
    # get the columns with NaN values in the specified row
    cols_with_nans =  user_data.iloc[row_idx][ user_data.iloc[row_idx].isnull()].index.tolist()

    # randomly fill 'n' of the NaN values in the specified row with values between 0 and 10
    random_values = np.random.randint(1, 3, n)
    for col in np.random.choice(cols_with_nans, n, replace=False):
        user_data.loc[row_idx, col] = random_values[n - 1]

In [139]:
user_data

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,5
1,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2,NaN,NaN,NaN,2,NaN,NaN,NaN
2,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5,NaN,2,NaN,4,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,1,4,4,4,4,4,4,1,...,1,1,1,1,1,1,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,5,NaN,5,4,5,5,5,5,4,1,...,NaN,1,1,1,1,NaN,NaN,NaN,1,1
71,2,2,2,2,NaN,NaN,2,2,2,2,...,NaN,2,2,2,2,2,2,2,NaN,NaN
72,1,1,NaN,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
73,NaN,2,NaN,2,NaN,NaN,NaN,NaN,2,NaN,...,2,5,2,5,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
user_data.to_csv("../data/user_data.csv", index=False)

In [141]:
matrix = user_data.div(user_data.sum(axis=1), axis=0)
# matrix = user_data

In [142]:
matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,268,279,284,293,298,305,307,335,336,359
0,0.026667,NaN,0.026667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.026667,NaN,NaN,0.026667,0.066667
1,NaN,NaN,NaN,NaN,NaN,0.055556,NaN,NaN,NaN,NaN,...,NaN,NaN,0.055556,NaN,NaN,NaN,0.055556,NaN,NaN,NaN
2,NaN,NaN,0.038462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.096154,NaN,0.038462,NaN,0.076923,NaN,0.038462
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.166667,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN
4,0.031008,0.03876,0.007752,0.031008,0.031008,0.031008,0.031008,0.031008,0.031008,0.007752,...,0.007752,0.007752,0.007752,0.007752,0.007752,0.007752,0.007752,0.007752,0.031008,0.007752


In [149]:
users_pierson_correlation = matrix.fillna(0).T.corr()

In [150]:
fig = px.imshow(users_pierson_correlation)
fig.show()

In [151]:
from sklearn.metrics.pairwise import cosine_similarity

user_cosine_similarity = cosine_similarity(matrix.fillna(0))
px.imshow(user_cosine_similarity)

In [152]:
picked_user_id = 1
users_pierson_correlation.drop(index= picked_user_id, inplace=True)
users_pierson_correlation.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,1.000000,-0.000259,0.058435,0.021677,-0.123506,0.030145,0.108520,-0.215973,-0.032838,0.133781,...,0.186721,0.028972,0.259866,0.172022,0.105049,-0.135798,0.027328,0.369833,-0.036415,-0.087668
2,0.058435,-0.164502,1.000000,0.200774,-0.023187,0.257128,0.213150,0.124301,-0.033229,0.138615,...,0.169653,0.146813,-0.143391,0.160065,0.403468,0.002863,-0.064714,0.060571,0.153644,0.018712
3,0.021677,-0.053249,0.200774,1.000000,-0.166050,0.186007,0.127791,0.312079,0.211123,0.317092,...,0.200847,0.376373,0.283046,0.198467,0.207232,-0.010809,0.018438,-0.052205,0.459188,-0.026129
4,-0.123506,-0.089450,-0.023187,-0.166050,1.000000,-0.087163,-0.036950,0.052062,-0.049572,-0.145426,...,-0.032907,-0.049048,-0.176452,0.126997,-0.059849,0.593916,-0.125991,-0.146969,-0.165219,0.634243
5,0.030145,0.093030,0.257128,0.186007,-0.087163,1.000000,0.100927,0.225166,-0.231916,0.174069,...,-0.045995,-0.022462,-0.059153,0.039041,-0.043976,-0.148998,0.070452,0.236032,0.128891,-0.076152


In [153]:
top_n_users = 10
user_similarity_threshold = 0.3
similar_users = users_pierson_correlation[users_pierson_correlation[picked_user_id] > user_similarity_threshold]

In [154]:
similar_users

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
16,0.239229,0.395147,-0.034192,-0.044186,-0.157324,0.026447,0.043867,0.004885,-0.067769,0.452644,...,0.021945,-0.077879,0.015242,-0.094335,-0.102031,-0.050387,0.274518,-0.134174,0.096800,-0.005403
25,-0.130499,0.339185,0.098570,0.066694,-0.120026,0.291896,0.163113,0.273512,0.106954,0.369188,...,-0.123264,-0.024944,-0.023007,0.052646,-0.233914,-0.233544,0.153702,-0.087894,-0.045921,-0.068737
28,-0.187037,0.315463,0.048993,0.037386,0.011438,0.187661,-0.081782,0.135122,0.347877,-0.079975,...,-0.111138,0.129332,0.147474,0.016349,0.048552,-0.137051,-0.020049,-0.098248,0.272653,-0.057167
54,0.244878,0.329745,0.183901,-0.016569,-0.239596,-0.083963,0.172170,0.066853,0.390043,0.214119,...,0.029095,0.178140,0.208618,0.079997,-0.048549,-0.250949,0.097299,-0.034959,0.118456,-0.111246


In [155]:
# Movies that the target user has watched
picked_userid_watched = matrix[matrix.index == picked_user_id].dropna(axis=1, how='all')
picked_userid_watched

,5,17,19,20,40,125,135,142,153,170,171,191,230,263,284,307
1,0.055556,0.055556,0.055556,0.055556,0.055556,0.111111,0.111111,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556


In [156]:
# Movies that similar users watched. Remove movies that none of the similar users have watched
similar_user_movies = matrix[matrix.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_movies

,0,1,3,4,5,6,7,8,9,10,...,268,279,284,293,298,305,307,335,336,359
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN
25,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,...,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.032895,0.013158,0.013158
28,NaN,0.013699,0.013699,0.013699,0.013699,0.013699,0.013699,NaN,0.013699,0.013699,...,NaN,0.068493,0.013699,NaN,0.013699,0.013699,0.013699,0.013699,0.013699,0.013699
54,NaN,0.013333,0.013333,NaN,NaN,0.013333,NaN,0.013333,0.013333,0.013333,...,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.013333,0.066667,0.013333,0.066667


In [157]:
# Remove the watched movie from the movie list
similar_user_movies.drop(picked_userid_watched.columns,axis=1, inplace=True, errors='ignore')
# Take a look at the data
similar_user_movies

,0,1,3,4,6,7,8,9,10,11,...,231,246,268,279,293,298,305,335,336,359
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN
25,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,...,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.013158,0.032895,0.013158,0.013158
28,NaN,0.013699,0.013699,0.013699,0.013699,0.013699,NaN,0.013699,0.013699,0.013699,...,NaN,0.013699,NaN,0.068493,NaN,0.013699,0.013699,0.013699,0.013699,0.013699
54,NaN,0.013333,0.013333,NaN,0.013333,NaN,0.013333,0.013333,0.013333,0.013333,...,0.013333,NaN,0.013333,0.013333,0.013333,0.013333,0.013333,0.066667,0.013333,0.066667


In [163]:
# A dictionary to store item scores
item_score = {}
# Loop through items
for i in similar_user_movies.columns:
  # Get the ratings for movie i
  movie_rating = similar_user_movies[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in similar_users.index:
    # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
      # Score is the sum of user similarity score multiply by the movie rating
      score = sum(similar_users[u] * movie_rating[u])
      # Add the score to the total score for the movie so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the item
  item_score[i] = total / count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
# Select top m movies
m = 10
ranked_item_score.head(m)

,movie,movie_score
50,305,0.109559
51,335,0.075628
53,359,0.062131
47,279,0.052910
10,12,0.052910
21,65,0.038670
32,128,0.027357
17,45,0.027357
25,84,0.027357
30,126,0.027357


In [176]:
cars_data.loc[cars_data.car_id.isin(ranked_item_score.movie) == True]

,car_id,car_model,used_label,price,exteriorColor,interiorColor,drivetrain,minMPG,maxMPG,fuelType,transmission,engine,mileage
0,1.0,Toyota Sienna SE,Used,$39,Red,Black,Front-wheel Drive,19.0,27.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,29403.0
2,3.0,RAM 1500 Laramie,Used,$41,Granite Crystal Clearcoat Metallic,Black,Four-wheel Drive,15.0,21.0,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,23173.0
3,4.0,Honda Accord Sport SE,Used,$28,Gray,–,Front-wheel Drive,29.0,35.0,Gasoline,Automatic CVT,1.5L I4 16V GDI DOHC Turbo,10598.0
5,6.0,Toyota 4Runner SR5,Used,$23,Black,–,Rear-wheel Drive,17.0,23.0,Gasoline,5-Speed Automatic,4.0L V6 24V MPFI DOHC,105469.0
6,7.0,Honda HR-V LX,Used,$20,Modern Steel Metallic,Black,Front-wheel Drive,28.0,34.0,Gasoline,Automatic CVT,1.8L I4 16V MPFI SOHC,10458.0
7,8.0,Mercedes-Benz E-Class E 350,Used,$18,Lunar Blue Metallic,Saffron,Rear-wheel Drive,21.0,30.0,Gasoline,7-Speed Automatic,3.5L V6 24V GDI DOHC,58157.0
8,9.0,Honda Pilot Touring 8-Passenger,Used,$44,Platinum White Pearl,Beige,All-wheel Drive,19.0,26.0,Gasoline,9-Speed Automatic,3.5L V6 24V GDI SOHC,14445.0
9,10.0,Dodge Charger Scat Pack,Dodge Certified,$46,Triple Nickel Clearcoat,Black,Rear-wheel Drive,15.0,24.0,Gasoline,8-Speed Automatic,6.4L V8 16V MPFI OHV,25642.0
10,11.0,Lexus RC 350 Base,Used,$42,Infrared,Silver,Rear-wheel Drive,19.0,28.0,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC,10236.0
11,12.0,Lexus GS 350 Base,Used,$16,Gray,Black,Rear-wheel Drive,19.0,28.0,Gasoline,6-Speed Automatic,3.5L V6 24V GDI DOHC,48605.0
